In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

from pandas_profiling import ProfileReport
import seaborn as sns
from sklearn import metrics
from scipy import stats

from copy import deepcopy

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,confusion_matrix
from sklearn.metrics import accuracy_score, mean_squared_error

In [ ]:
train_df = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
test_df = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
sub_df = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
shops_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
items_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
item_categories_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
shops_df.head()

In [ ]:
items_df.head()

In [ ]:
train_df.describe()

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.value_counts

In [ ]:
train_df.shape

In [ ]:
train_df.drop(['date_block_num','item_price'], axis=1, inplace=True)

In [ ]:
train_df['date'] = pd.to_datetime(train_df['date'], dayfirst=True)
train_df['date'] = train_df['date'].apply(lambda x: x.strftime('%Y-%m'))
train_df.head()

In [ ]:
df = train_df.groupby(['date','shop_id','item_id']).sum()
df = df.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)
df.reset_index(inplace=True)
df.head()

In [ ]:
test_df = pd.merge(test_df, df, on=['shop_id','item_id'], how='left')
test_df.drop(['ID', '2013-01'], axis=1, inplace=True)
test_df = test_df.fillna(0)
test_df.head()

In [ ]:
Y_train = df['2015-10'].values
X_train = df.drop(['2015-10'], axis = 1)
X_test = test_df

print(X_train.shape, Y_train.shape)
print(X_test.shape)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split( X_train, Y_train, test_size=0.2, random_state=101)
print ('Train set:', x_train.shape,  y_train.shape)
print ('Test set:', x_test.shape,  y_test.shape)

Simple Regressor

In [ ]:
param_grid = {
    'n_estimators': [5, 10, 15, 20],
    'max_depth': [2, 5, 7, 9]
}



# Instantiate model with 100 decision trees
clf = XGBRegressor(random_state = 42)

clf.fit(x_train, y_train)

In [ ]:
# Use the forest's predict method on the test data
predictions = clf.predict(x_test)

# Calculate the absolute errors
errors = abs(predictions - y_test)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
preds=pd.Series(clf.predict(test_df))

In [ ]:
submission = pd.DataFrame({
    "ID": test_df.index, 
    "item_cnt_month": preds
})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('xgb_submission.csv', index=False)